<a href="https://colab.research.google.com/github/sureshkumaroffl/AmazonNLP_Task/blob/main/Amazon_NLP_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [356]:
!pip install pyspellchecker

import pandas as pd
from nltk import word_tokenize
import nltk
nltk.download('punkt')
from spellchecker import SpellChecker
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [357]:
DataSet=pd.read_csv('https://raw.githubusercontent.com/sureshkumaroffl/AmazonNLP_Task/main/amazondata1.csv')

In [358]:
DataSet.head()

,Unnamed: 0,Helpful Votes (bin),Number of Records,Star Rating (bin),Customer Id,Helpful Votes,Overall Votes,Product Id,Review Body,Review Year,Review Headline,Star Rating
0,0,0.0,1,0.0,NaN,4.0,14.0,26009102,You will love this book. It is a hard long re...,03/17/2005 0:00,Best Book Ever,5.0
1,1,NaN,1,NaN,NaN,NaN,NaN,7491727,This is the UK edition of Dr. Omit's book. Dr....,NaN,researchers from John Hopkins School of Medici...,NaN
2,2,0.0,1,0.0,NaN,2.0,2.0,002782683X,This is a fun and entertaining book about lear...,06/25/2012 0:00,Michelle,5.0
3,3,0.0,1,0.0,NaN,0.0,0.0,60187271,"Started a big slow, but once into it the autho...",06/09/2013 0:00,Loved the book,5.0
4,4,0.0,1,0.0,NaN,14.0,20.0,60392452,Received this book as a Christmas present. I h...,08/05/2003 0:00,Challenges your assumptions,4.0


In [359]:
DataSet.dropna(axis=0,inplace=True)
DataSet.shape

(12918, 12)

In [361]:
DataSet['Review_status']=''
DataSet.loc[DataSet['Star Rating']>=3,'Review_status']='Good'
# DataSet.loc[DataSet['Star Rating']==3,'Review_status']='Neutral'
DataSet.loc[DataSet['Star Rating']<3,'Review_status']='Bad'

##LowerCasing

In [362]:
def df_lower(df):
  l_df=df.str.lower()
  return l_df

In [363]:
DataSet['Review Body']=df_lower(DataSet['Review Body'])

In [364]:
DataSet['Review Body']

76       john green wrote an amazing book. worth the mo...
77          classic. awesome. american traditional tattoo.
78                                              great book
79                                          little helpful
80       got this for grand baby and she is not potty t...
                               ...                        
22400         perfect and a lot faster than they quoted :)
22401    for the cheap price, thats exactly what these ...
22402    i got this for my 18 month old daughter.  she ...
22403    this book is a guide to being gentle with your...
22404          loved the entire series.  very imaginative.
Name: Review Body, Length: 12918, dtype: object

##Extra Space removing

In [365]:
def df_space_remove(text):
  return " ".join(text.split())

In [366]:
DataSet['Review Body']=DataSet['Review Body'].apply(df_space_remove)

##Tokenization

In [367]:
DataSet['Review Body']=DataSet['Review Body'].apply(word_tokenize)

In [368]:
DataSet['Review Body']

76       [john, green, wrote, an, amazing, book, ., wor...
77       [classic, ., awesome, ., american, traditional...
78                                           [great, book]
79                                       [little, helpful]
80       [got, this, for, grand, baby, and, she, is, no...
                               ...                        
22400    [perfect, and, a, lot, faster, than, they, quo...
22401    [for, the, cheap, price, ,, thats, exactly, wh...
22402    [i, got, this, for, my, 18, month, old, daught...
22403    [this, book, is, a, guide, to, being, gentle, ...
22404    [loved, the, entire, series, ., very, imaginat...
Name: Review Body, Length: 12918, dtype: object

##Spelling Correction

In [369]:
def spell_check(text):

    result = []
    spell = SpellChecker()
    for word in text:
        correct_word = spell.correction(word)
        result.append(correct_word)

    return result

#Test
text = "i like india godd vry mch".split()
spell_check(text)

['i', 'like', 'india', 'good', 'very', 'much']

In [370]:
# DataSet['Review Body']=DataSet['Review Body'].apply(spell_check)

##Stopwords Removing

In [371]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [372]:
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)

    return result

#Test
text = "this is the only solution of that question".split()
print(remove_stopwords(text))

['solution', 'question']


In [373]:
DataSet['Review Body']=DataSet['Review Body'].apply(remove_stopwords)

In [374]:
DataSet['Review Body']

76       [john, green, wrote, amazing, book, ., worth, ...
77       [classic, ., awesome, ., american, traditional...
78                                           [great, book]
79                                       [little, helpful]
80       [got, grand, baby, potty, training, ., good, b...
                               ...                        
22400                 [perfect, lot, faster, quoted, :, )]
22401    [cheap, price, ,, thats, exactly, cards, ., li...
22402    [got, 18, month, old, daughter, ., loves, book...
22403    [book, guide, gentle, still, moving, forward, ...
22404           [loved, entire, series, ., imaginative, .]
Name: Review Body, Length: 12918, dtype: object

##Punctuations Removing

In [375]:
def remove_punct(text):

    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

#Test
text=['@','Amazon', 'tools', 'kids', 'dysfunction','$','!']
print(text)
remove_punct(text)

['@', 'Amazon', 'tools', 'kids', 'dysfunction', '$', '!']


['Amazon', 'tools', 'kids', 'dysfunction']

In [376]:
DataSet['Review Body']=DataSet['Review Body'].apply(remove_punct)

##Frequent Words Removing

In [377]:
def frequent_words(df):

    lst=[]
    for text in df.values:
        lst+=text
    fdist=FreqDist(lst)
    return fdist.most_common(2)

frequent_words(DataSet['Review Body'])

[('book', 11111), ('br', 8176)]

In [378]:
freq_words = frequent_words(DataSet['Review Body'])

lst = []
for a,b in freq_words:
    lst.append(b)

def remove_freq_words(text):

    result=[]
    for item in text:
        if item not in lst:
            result.append(item)

    return result

DataSet['Review Body']=DataSet['Review Body'].apply(remove_freq_words)

##Lemmatization

In [379]:
def lemmatization(text):

    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()

        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'

        result.append(wordnet.lemmatize(token,pos))

    return result

#Test
text = ['running','ran','runs']
lemmatization(text)

['run', 'ran', 'run']

In [380]:
DataSet['Review Body']=DataSet['Review Body'].apply(lemmatization)

##Stemming

In [381]:
def stemming(text):
    porter = PorterStemmer()

    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result

#Test
text=['Connects','Connecting','Connections','Connected','Connection','Connectings','Connect']
stemming(text)

['connect', 'connect', 'connect', 'connect', 'connect', 'connect', 'connect']

In [382]:
DataSet['Review Body']=DataSet['Review Body'].apply(stemming)

##List to String

In [385]:
def list_to_string(text):
  listTxt=""
  for str_val in text:
    listTxt +=str_val+''
  # str_val=''.join(map(str,text))
  return listTxt

In [264]:
# DataSet['Review Body']=DataSet['Review Body'].apply(list_to_string)

In [171]:
print(list_to_string(['mayava','krishna']))

mayavakrishna


In [386]:
DataSet

,Unnamed: 0,Helpful Votes (bin),Number of Records,Star Rating (bin),Customer Id,Helpful Votes,Overall Votes,Product Id,Review Body,Review Year,Review Headline,Star Rating,Review_status
76,76,0.0,1,0.0,10001274.0,0.0,0.0,142402516,"[john, green, write, amaz, book, worth, money]",08/31/2015 0:00,Amazing.,5.0,Good
77,77,0.0,1,0.0,10001635.0,0.0,1.0,945367163,"[classic, awesom, american, tradit, tattoo]",08/29/2015 0:00,Awesome. American traditional tattoo,5.0,Good
78,78,0.0,1,0.0,10002164.0,0.0,0.0,1512081140,"[great, book]",08/28/2015 0:00,Five Stars,5.0,Good
79,79,0.0,1,0.0,10002164.0,0.0,0.0,B005DI89EI,"[littl, help]",08/28/2015 0:00,One Star,1.0,Bad
80,80,0.0,1,0.0,10002870.0,0.0,0.0,764152319,"[get, grand, babi, potti, train, good, book]",08/28/2015 0:00,Very good book.,5.0,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22400,22400,0.0,1,0.0,47830325.0,0.0,1.0,1742200117,"[perfect, lot, faster, quot]",08/28/2015 0:00,Four Stars,4.0,Good
22401,22401,0.0,1,0.0,47830817.0,0.0,0.0,769647197,"[cheap, price, that, exactli, card, like, deck...",08/28/2015 0:00,Good for the price,4.0,Good
22402,22402,0.0,1,0.0,47830817.0,2.0,2.0,1575425009,"[get, 18, month, old, daughter, love, book, ni...",08/27/2015 0:00,"Great book, great lesson for kids!",5.0,Good
22403,22403,0.0,1,0.0,47830878.0,1.0,1.0,1940014417,"[book, guid, gentl, still, move, forward, life...",08/31/2015 0:00,"Be gentle, be aware, move forward!",5.0,Good


##ML

In [387]:
Independent_var =DataSet['Review_status']
Dependent_var = DataSet['Review Body']

IV_train, IV_test, DV_train, DV_test = train_test_split(Independent_var, Dependent_var, random_state = 225)

In [388]:
tvec = TfidfVectorizer()
clf2 = LogisticRegression(max_iter=10000)

In [389]:
print('IV_train :', len(IV_train))
print('IV_test  :', len(IV_test))
print('DV_train :', len(DV_train))
print('DV_test  :', len(DV_test))

IV_train : 9688
IV_test  : 3230
DV_train : 9688
DV_test  : 3230


In [ ]:
model = Pipeline([('vectorizer',tvec),('classifier',clf2)])
model.fit(IV_train, DV_train )
predictions = model.predict(IV_test)
confusion_matrix(predictions, DV_test)

In [353]:
example = ["we had a corporate outing to the etc hotel,we had and average experince it but the roads to reach the destination was so good"]
ex2=["i had a bad day in the hotel"]
ex3=['i was very sad that there were no pictures']
result = model.predict(example)
result

array(['Good'], dtype=object)